Vsakemu študentu so pulz izmerili dvakrat. Določeni
so imeli med obema meritvama fizično obremenitev (tek na mestu), določeni ne.
Podatki so zbrani v tabeli Pulz. 

Raziščite, katere od dejavnikov višina, teža, spol in vadba je smiselno vključiti
v linearni model, ki bo opisoval odvisnost prve meritve pulza od teh dejavnikov: poiščite model z najmanjšo Akaikejevo informacijo. Slednja nam pomaga izbrati le bistvene dejavnike. Akaikejeva informacija je sicer definirana
z verjetjem, a pri linearni regresiji in Gaussovem modelu je le-ta ekvivalentna
naslednji modifikaciji: $$AIC :=  2m + n\cdot ln(RSS) $$ kjer je m število parametrov, n pa je število opažanj.

In [69]:
import pandas as pd
import numpy as np
import itertools
from itertools import *
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [52]:
data = pd.read_csv("Pulz.csv")
data.head()


,VISINA,TEZA,STAROST,SPOL,KADI,ALKOHOL,VADBA,OBREMENITEV,PULZ1,PULZ2,LETO
0,173,57.0,18,2,2,1,2,2,86,88,93
1,179,58.0,19,2,2,1,2,1,82,150,93
2,167,62.0,18,2,2,1,1,1,96,176,93
3,195,84.0,18,1,2,1,1,2,71,73,93
4,173,64.0,18,2,2,1,3,2,90,88,93


In [29]:
data.columns
my_data = data.iloc[:,[0,1,3,6]]
my_data.head()
my_data.describe()

,VISINA,TEZA,SPOL,VADBA
count,109.000000,109.000000,109.000000,109.000000
mean,171.568807,66.353211,1.458716,2.211009
std,16.150546,15.225208,0.500594,0.653670
min,68.000000,27.000000,1.000000,1.000000
25%,165.000000,56.000000,1.000000,2.000000
50%,172.000000,63.000000,1.000000,2.000000
75%,180.000000,75.000000,2.000000,3.000000
max,195.000000,110.000000,2.000000,3.000000


In [53]:
data.corr()

,VISINA,TEZA,STAROST,SPOL,KADI,ALKOHOL,VADBA,OBREMENITEV,PULZ1,PULZ2,LETO
VISINA,1.000000,0.579895,0.025570,-0.415088,-0.037404,-0.257844,-0.181624,0.026767,-0.278283,-0.086510,-0.123301
TEZA,0.579895,1.000000,0.148881,-0.601552,-0.050473,-0.198814,-0.213634,-0.041983,-0.194666,-0.029249,0.048198
STAROST,0.025570,0.148881,1.000000,-0.124119,-0.067895,0.216817,0.111148,0.090770,-0.145909,-0.144600,0.124160
SPOL,-0.415088,-0.601552,-0.124119,1.000000,0.125050,0.141261,0.182499,-0.033517,0.126016,0.062162,-0.007722
KADI,-0.037404,-0.050473,-0.067895,0.125050,1.000000,0.140094,-0.125398,-0.101273,-0.047012,0.083144,-0.004151
ALKOHOL,-0.257844,-0.198814,0.216817,0.141261,0.140094,1.000000,0.264736,0.104003,0.002992,-0.065705,0.055837
VADBA,-0.181624,-0.213634,0.111148,0.182499,-0.125398,0.264736,1.000000,0.077266,0.210035,0.022724,0.103633
OBREMENITEV,0.026767,-0.041983,0.090770,-0.033517,-0.101273,0.104003,0.077266,1.000000,0.014948,-0.817154,-0.103226
PULZ1,-0.278283,-0.194666,-0.145909,0.126016,-0.047012,0.002992,0.210035,0.014948,1.000000,0.361480,0.051052
PULZ2,-0.086510,-0.029249,-0.144600,0.062162,0.083144,-0.065705,0.022724,-0.817154,0.361480,1.000000,0.088176


In [47]:
kombinacije = [] #vse mnozice podatkov ki jih bomo uporabili v modelu
stolpci = [0,1, 2, 3]
for L in range(0, len(stolpci)+1):
    for subset in itertools.combinations(stolpci, L):
        kombinacije.append(list(subset))
del kombinacije[0] #izbrisemo prazno mnozico
print(kombinacije)


[[0], [1], [2], [3], [0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3], [0, 1, 2, 3]]


In [106]:
n_modelov = len(kombinacije)
AIC = []
reg = LinearRegression()
Y = data.PULZ1 #ocenjujemo ta podatek
n = len(Y)
#za vsako podmnozico vhodnih parametrov bomo evaluirali model
for kombinacija in kombinacije:
    m = len(kombinacija) #st parametrov
    reg.fit(my_data.iloc[: , kombinacija], Y) #regresija
    #print(reg.coef_) #koeficienti regresije
    
    Y_ocenjeno = reg.predict(my_data.iloc[: , kombinacija])
    mse = mean_squared_error(Y, Y_ocenjeno)
    # rss = N* MSE
    rss = n * mse 
    aic = 2 * m + n * np.log(rss) 
    AIC.append(aic) #seznam AIC-jev za posamezni model
    


[-0.22912646]
[-0.17002057]
[3.34745763]
[4.27276342]
[-0.20517826 -0.0438074 ]
[-0.22478925  0.3371105 ]
[-0.20446212  3.35524476]
[-0.16268106  0.37108364]
[-0.13708715  3.59062538]
[2.40948639 3.93601015]
[-0.20616665 -0.04951805 -0.31946873]
[-0.19355474 -0.02067777  3.3012999 ]
[-0.20731686 -0.23318727  3.37502473]
[-0.13758522 -0.02569853  3.59173868]
[-0.19548019 -0.03214025 -0.65117012  3.3266313 ]


In [154]:
min_index = AIC.index(min(AIC)) #index z parametri ki dajo najmanjši AIC
optimalni_podatki = my_data.iloc[:,kombinacije[6]]


optimalni_AIC = min(AIC)
print("Najmanjši AIC je: ", optimalni_AIC)
optimalni_podatki.head()

Najmanjši AIC je:  1066.5074099762016


,VISINA,VADBA
0,173,2
1,179,2
2,167,1
3,195,1
4,173,3


Za vsakega od dejavnikov kajenje in alkohol vaš model preizkusite proti širšemu
modelu, ki poleg dejavnikov, izbranih v prejšnji točki, vključi tudi še dodatni
dejavnik.


In [161]:
kajenje = data.iloc[:,[4]]
alkohol = data.iloc[:,[5]]

dodatno_kadi = pd.concat([optimalni_podatki, kajenje], axis=1)
dodatno_kadi.head()

dodatno_alkohol = pd.concat([optimalni_podatki, alkohol], axis=1)
dodatno_alkohol.head()


,VISINA,VADBA,ALKOHOL
0,173,2,1
1,179,2,1
2,167,1,1
3,195,1,1
4,173,3,1


In [162]:
kajenje = data.iloc[:,[4]]
print(kajenje.value_counts())
alkohol = data.iloc[:,[5]]
print(alkohol.value_counts())

KADI
2       98
1       11
dtype: int64
ALKOHOL
1          67
2          42
dtype: int64


In [163]:
#dimenzije vseh podatkov enake tako d lahko m in n parameter ostaneta za vse
m = dodatno_alkohol.shape[0] #velikost vzorca
n = dodatno_alkohol.shape[1] #st parametrov

AIC_2 = [optimalni_AIC]
dic = {}
dic[optimalni_AIC] = optimalni_podatki

Y = data.PULZ1 #ocenjujemo ta podatek  
reg.fit(dodatno_alkohol, Y) #regresija
Y_ocenjeno = reg.predict(dodatno_alkohol)
mse = mean_squared_error(Y, Y_ocenjeno)
    # rss = N* MSE
rss = n * mse 
aic = 2 * m + n * np.log(rss) 
AIC_2.append(aic) #seznam AIC-jev za posamezni model
dic[aic] = dodatno_alkohol

reg.fit(dodatno_kadi, Y) #regresija
Y_ocenjeno = reg.predict(dodatno_kadi)
mse = mean_squared_error(Y, Y_ocenjeno)
    # rss = N* MSE
rss = n * mse 
aic = 2 * m + n * np.log(rss) 
AIC_2.append(aic) #seznam AIC-jev za posamezni model
dic[aic] = dodatno_kadi

print(AIC_2)

[1066.5074099762016, 236.42333708010682, 236.4607823819059]


In [164]:
min_aic = min(AIC_2)
print("Najmanjši AIC je: ", min_aic)

print("Od prejšnjega najmanjšega AIC je manjši za: ", abs(optimalni_AIC - min_aic))

print("Optimalni podatki so : ")
dic[min_aic]


Najmanjši AIC je:  236.42333708010682
Od prejšnjega najmanjšega AIC je manjši za:  830.0840728960948
Optimalni podatki so : 


,VISINA,VADBA,ALKOHOL
0,173,2,1
1,179,2,1
2,167,1,1
3,195,1,1
4,173,3,1
...,...,...,...
104,93,3,2
105,161,3,2
106,182,3,1
107,170,1,1


In [166]:
list.sort(AIC_2)
diff = AIC_2[1] - AIC_2[0]
diff

0.037445301799067465

Vidimo, da dodajanje vsakega od dodatnih podatkov zelo pripomore k zmanjšanju Akaikejeve informacije pri čemer je za  zanemarljivo razliko 0.03744 boljše dodajanje alkohola. 